In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -r "/content/drive/MyDrive/train.zip" /content
!cp -r /content/drive/MyDrive/val.zip /content
!unzip "train.zip" -d /content
!unzip /content/val -d /content

Streaming output truncated to the last 5000 lines.
  inflating: /content/val/n008890/0007_01.jpg  
  inflating: /content/val/n008890/0009_01.jpg  
  inflating: /content/val/n008890/0011_01.jpg  
  inflating: /content/val/n008890/0012_01.jpg  
  inflating: /content/val/n008890/0013_01.jpg  
  inflating: /content/val/n008890/0016_01.jpg  
  inflating: /content/val/n008890/0017_01.jpg  
  inflating: /content/val/n008890/0018_01.jpg  
  inflating: /content/val/n008890/0026_01.jpg  
  inflating: /content/val/n008890/0027_01.jpg  
  inflating: /content/val/n008890/0031_01.jpg  
  inflating: /content/val/n008890/0032_01.jpg  
  inflating: /content/val/n008890/0034_01.jpg  
  inflating: /content/val/n008890/0036_01.jpg  
  inflating: /content/val/n008890/0042_01.jpg  
  inflating: /content/val/n008890/0043_01.jpg  
  inflating: /content/val/n008890/0046_01.jpg  
  inflating: /content/val/n008890/0047_01.jpg  
  inflating: /content/val/n008890/0049_01.jpg  
  inflating: /content/val/n008890/005

In [3]:
!git clone https://github.com/deepinsight/insightface.git

Cloning into 'insightface'...
remote: Enumerating objects: 12592, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 12592 (delta 104), reused 89 (delta 89), pack-reused 12444 (from 3)
Receiving objects: 100% (12592/12592), 58.46 MiB | 16.67 MiB/s, done.
Resolving deltas: 100% (6498/6498), done.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import math
from tqdm import tqdm
import numpy as np
import itertools
from collections import defaultdict
import os
from PIL import Image
from pathlib import Path

import sys
import arc_scores
import helper
import evaluate
from models XiaoYing

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [18]:
IMAGE_SIZE = 112
BATCH_SIZE = 256
NUM_EPOCHS = 10
FEATURE_DIM = 512

base_lr_backbone = 0.1
base_lr_margin = 0.5
weight_decay = 5e-4

step_milestones = [5, 10, 15]
step_gamma = 0.1

In [6]:
class FastImageFolder(Dataset):
    def __init__(self, root, transform=None, extensions=('.jpg', '.jpeg', '.png', '.bmp')):
        self.root = Path(root)
        self.transform = transform
        self.extensions = extensions

        # Scan nhanh và cache paths
        self.samples = []
        self.classes = sorted([d.name for d in self.root.iterdir() if d.is_dir()])
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

        print(f"Scanning {root}...")
        for class_name in self.classes:
            class_dir = self.root / class_name
            class_idx = self.class_to_idx[class_name]

            for ext in self.extensions:
                for img_path in class_dir.glob(f'*{ext}'):
                    self.samples.append((str(img_path), class_idx))

        print(f"Found {len(self.samples)} images in {len(self.classes)} classes")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, target = self.samples[idx]
        image = Image.open(path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, target

In [7]:
train_transforms = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transforms = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [21]:
train_folder = 'train'
train_dataset = FastImageFolder(train_folder, transform=train_transforms)

test_folder = 'val'
test_dataset = FastImageFolder(test_folder, transform=test_transforms)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)

NUM_CLASSES = len(train_dataset.classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpus = torch.cuda.device_count()

print("="*60)
print(f"MULTI-GPU TRAINING SETUP")
print("="*60)
print(f"GPUs available: {n_gpus}")
for i in range(n_gpus):
    print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
print(f"Dataset: {NUM_CLASSES} classes, {len(train_dataset)} images")
print(f"Batch size: {BATCH_SIZE} (effective: {BATCH_SIZE})")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Steps per epoch: {len(train_loader)}")
print("="*60)

Scanning train...
Found 1119807 images in 5115 classes
Scanning val...
Found 114964 images in 555 classes
MULTI-GPU TRAINING SETUP
GPUs available: 1
  GPU 0: NVIDIA L4
Dataset: 5115 classes, 1119807 images
Batch size: 256 (effective: 256)
Epochs: 10
Steps per epoch: 4375


In [9]:
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, s=64.0, m=0.5):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m

        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, inputs, labels):
        cosine = F.linear(
            F.normalize(inputs, p=2, dim=1),
            F.normalize(self.weight, p=2, dim=1)
        )
        sine = torch.sqrt(torch.clamp(1.0 - torch.pow(cosine, 2), 1e-9, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        phi = torch.where(cosine > self.th, phi, cosine - self.mm)

        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, labels.view(-1, 1), 1.0)

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output

In [10]:
def step_lr(optimizer, base_lr_backbone, base_lr_margin, epoch,
            milestones=[10, 15], gamma=0.1):
    lr_scale = 1.0
    for milestone in milestones:
        if epoch >= milestone:
            lr_scale *= gamma

    lr_backbone = base_lr_backbone * lr_scale
    lr_margin = base_lr_margin * lr_scale

    optimizer.param_groups[0]["lr"] = lr_backbone
    optimizer.param_groups[1]["lr"] = lr_margin

    return lr_backbone, lr_margin

In [11]:
class EarlyStopping:
    def __init__(self, patience=5, epsilon=0.001, save_path="best.pt"):
        self.patience = patience
        self.save_path = save_path
        self.epsilon = epsilon
        self.best_acc = -1
        self.counter = 0
        self.should_stop = False

    def step(self, val_acc, model, margin):
        if val_acc > self.best_acc + self.epsilon:
            self.best_acc = val_acc
            self.counter = 0

            model_to_save = model.module if hasattr(model, 'module') else model
            margin_to_save = margin.module if hasattr(margin, 'module') else margin

            checkpoint = {
                'model_state_dict': model.state_dict(),
                'margin_state_dict': margin.state_dict(),
                'best_tar_far': self.best_acc
            }
            torch.save(checkpoint, self.save_path)
            print(f"✓ Saved best model: TAR@FAR1e-3={val_acc:.4f}")
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.should_stop = True
                print("⚠ Early stopping triggered!")

In [ ]:
model = XiaoYing().to(device)
margin = ArcMarginProduct(
    in_features=FEATURE_DIM,
    out_features=NUM_CLASSES,
    s=64.0,
    m=0.3
).to(device)

if n_gpus > 1:
    print(f"Using DataParallel with {n_gpus} GPUs")
    model = nn.DataParallel(model)
    margin = nn.DataParallel(margin)

model = model.to(device)
margin = margin.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD([
    {"params": model.parameters(), "lr": base_lr_backbone},
    {"params": margin.parameters(), "lr": base_lr_margin}
], momentum=0.9, weight_decay=5e-4)

steps_per_epoch = len(train_loader)

In [ ]:
def train():
    train_losses = []
    test_losses = []
    tarfar3s = []
    tarfar4s = []
    accs = []
    rocs = []

    # early stopping
    early = EarlyStopping(patience=3, epsilon=0.001)

    for epoch in range(NUM_EPOCHS):
        model.train()
        margin.train()

        train_loss = []

        lr_backbone, lr_margin = step_lr(
            optimizer, base_lr_backbone, base_lr_margin,
            epoch, milestones=step_milestones, gamma=step_gamma
        )
        # bar config
        pbar = tqdm(
            enumerate(train_loader),
            total=len(train_loader),
            desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [LR: {lr_backbone:.6f}]"
        )

        for step, (inputs, targets) in pbar:
            inputs = inputs.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()

            # Forward
            outputs = model(inputs)
            outputs = F.normalize(outputs, p=2, dim=1)
            logits = margin(outputs, targets)
            loss = criterion(logits, targets)

            # Backward
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            torch.nn.utils.clip_grad_norm_(margin.parameters(), 5.0)
            optimizer.step()

            train_loss.append(loss.item())
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})

        train_loss = np.mean(train_loss)

        # ==================== EVALUATION ====================
        model.eval()
        margin.eval()

        test_loss = []
        embs = []
        labels = []

        with torch.no_grad():
            for inputs, targets in tqdm(test_loader, desc="Evaluating"):
                inputs = inputs.to(device)
                targets = targets.to(device)

                outputs = model(inputs)
                outputs = F.normalize(outputs, p=2, dim=1)
                logits = margin(outputs, targets)
                loss = criterion(logits, targets)
                test_loss.append(loss.item())

                embs.append(outputs.cpu())
                labels.append(targets.cpu())

        test_loss = np.mean(test_loss)
        eval_res = evaluate.evaluate(embs, labels, max_per_class=50, n_linspace=1000)

        # Get current learning rates
        current_lr_backbone = optimizer.param_groups[0]['lr']
        current_lr_margin = optimizer.param_groups[1]['lr']

        early.step(eval_res["tar_far_3"], model, margin)

        # helper function
        helper.save_checkpoint(model, margin, epoch, optimizer, train_loss, test_loss, eval_res["tar_far_3"], eval_res["tar_far_4"])
        helper.print_results(optimizer, epoch, NUM_EPOCHS, train_loss,test_loss, eval_res)

        # save scores
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        tarfar3s.append(eval_res["tar_far_3"])
        tarfar4s.append(eval_res["tar_far_4"])
        accs.append(eval_res["accuracy"])
        rocs.append(eval_res["roc_auc"])

        if early.should_stop:
            print("Training stopped early.")
            break

    print(f"\nTraining completed! Best TAR@FAR: {early.best_acc:.4f}")
    return train_losses, test_losses, tarfar3s, tarfar4s, accs, rocs

In [ ]:
train_losses, test_losses, tarfar3s, tarfar4s, accs, rocs = train()

Evaluating: 100%|██████████| 450/450 [02:10<00:00,  3.45it/s]


✓ Saved best model: TAR@FAR1e-3=0.5341
Saved checkpoint at epoch 1

Epoch 1/4
Learning Rate: Backbone=0.100000, Margin=0.500000
Train Loss: 18.7305
Test Loss: 47.3358
Eval Metrics:
  - Accuracy: 0.9063
  - ROC AUC: 0.9661
  - TAR@FAR1e-3: 0.5341
  - TAR@FAR1e-4: 0.3824
  - Threshold: 0.2722



Evaluating: 100%|██████████| 450/450 [02:10<00:00,  3.44it/s]


✓ Saved best model: TAR@FAR1e-3=0.6387
Saved checkpoint at epoch 2

Epoch 2/4
Learning Rate: Backbone=0.100000, Margin=0.500000
Train Loss: 9.9495
Test Loss: 45.4572
Eval Metrics:
  - Accuracy: 0.9263
  - ROC AUC: 0.9761
  - TAR@FAR1e-3: 0.6387
  - TAR@FAR1e-4: 0.4694
  - Threshold: 0.2455



Epoch 3/4 [LR: 0.100000]:  17%|█▋        | 765/4375 [13:24<1:03:15,  1.05s/it, loss=10.0397]

In [14]:
def load_checkpoint(path):
    start_epoch = 0
    resume_path = path

    if os.path.exists(resume_path):
        print(f"==> Loading checkpoint from {resume_path}")
        checkpoint = torch.load(resume_path, map_location=device, weights_only=False)

        model_to_load = model.module if hasattr(model, 'module') else model
        margin_to_load = margin.module if hasattr(margin, 'module') else margin

        model_to_load.load_state_dict(checkpoint['model_state_dict'])
        margin_to_load.load_state_dict(checkpoint['margin_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        start_epoch = checkpoint['epoch'] + 1
        print(f"==> Resuming from epoch {start_epoch}")

    return start_epoch

In [16]:
def train():
    train_losses = []
    test_losses = []
    tarfar3s = []
    tarfar4s = []
    accs = []
    rocs = []

    # early stopping
    early = EarlyStopping(patience=3, epsilon=0.001)

    start =  load_checkpoint('/content/checkpoint_epoch_2.pt')

    for epoch in range(start, NUM_EPOCHS):
        model.train()
        margin.train()

        train_loss = []

        lr_backbone, lr_margin = step_lr(
            optimizer, base_lr_backbone, base_lr_margin,
            epoch, milestones=step_milestones, gamma=step_gamma
        )
        # bar config
        pbar = tqdm(
            enumerate(train_loader),
            total=len(train_loader),
            desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [LR: {lr_backbone:.6f}]"
        )

        for step, (inputs, targets) in pbar:
            inputs = inputs.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()

            # Forward
            outputs = model(inputs)
            outputs = F.normalize(outputs, p=2, dim=1)
            logits = margin(outputs, targets)
            loss = criterion(logits, targets)

            # Backward
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            torch.nn.utils.clip_grad_norm_(margin.parameters(), 5.0)
            optimizer.step()

            train_loss.append(loss.item())
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})

        train_loss = np.mean(train_loss)

        # ==================== EVALUATION ====================
        model.eval()
        margin.eval()

        test_loss = []
        embs = []
        labels = []

        with torch.no_grad():
            for inputs, targets in tqdm(test_loader, desc="Evaluating"):
                inputs = inputs.to(device)
                targets = targets.to(device)

                outputs = model(inputs)
                outputs = F.normalize(outputs, p=2, dim=1)
                logits = margin(outputs, targets)
                loss = criterion(logits, targets)
                test_loss.append(loss.item())

                embs.append(outputs.cpu())
                labels.append(targets.cpu())

        test_loss = np.mean(test_loss)
        eval_res = evaluate.evaluate(embs, labels, max_per_class=50, n_linspace=1000)

        # Get current learning rates
        current_lr_backbone = optimizer.param_groups[0]['lr']
        current_lr_margin = optimizer.param_groups[1]['lr']

        early.step(eval_res["tar_far_3"], model, margin)

        # helper function
        helper.save_checkpoint(model, margin, epoch, optimizer, train_loss, eval_res["tar_far_3"], eval_res["tar_far_4"])
        helper.print_results(optimizer, epoch, NUM_EPOCHS, train_loss, eval_res)

        # save scores
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        tarfar3s.append(eval_res["tar_far_3"])
        tarfar4s.append(eval_res["tar_far_4"])
        accs.append(eval_res["accuracy"])
        rocs.append(eval_res["roc_auc"])

        if early.should_stop:
            print("Training stopped early.")
            break

    print(f"\nTraining completed! Best TAR@FAR: {early.best_acc:.4f}")
    return train_losses, test_losses, tarfar3s, tarfar4s, accs, rocs

train()

==> Loading checkpoint from /content/checkpoint_epoch_2.pt
==> Resuming from epoch 2


Evaluating: 100%|██████████| 450/450 [02:10<00:00,  3.45it/s]


✓ Saved best model: TAR@FAR1e-3=0.6645
Saved checkpoint at epoch 3

Epoch 3/4
Learning Rate: Backbone=0.100000, Margin=0.500000
Train Loss: 8.7083
Eval Metrics:
  - Accuracy: 0.9316
  - ROC AUC: 0.9789
  - TAR@FAR1e-3: 0.6645
  - TAR@FAR1e-4: 0.5094
  - Threshold: 0.2456



Evaluating: 100%|██████████| 450/450 [02:10<00:00,  3.45it/s]


Saved checkpoint at epoch 4

Epoch 4/4
Learning Rate: Backbone=0.100000, Margin=0.500000
Train Loss: 8.2046
Eval Metrics:
  - Accuracy: 0.9229
  - ROC AUC: 0.9737
  - TAR@FAR1e-3: 0.6344
  - TAR@FAR1e-4: 0.4719
  - Threshold: 0.2532


Training completed! Best TAR@FAR: 0.6645


([np.float64(8.708303947012766), np.float64(8.204615426090786)],
 [np.float64(45.418582611083984), np.float64(44.79844914754232)],
 [0.6644632957800155, 0.6344437830228673],
 [0.5093878370972816, 0.4718556912700722],
 [0.9316314939740884, 0.9229157465439082],
 [0.9789373626703863, 0.973663234659881])

In [22]:
def train():
    train_losses = []
    test_losses = []
    tarfar3s = []
    tarfar4s = []
    accs = []
    rocs = []

    # early stopping
    early = EarlyStopping(patience=3, epsilon=0.001)

    start =  load_checkpoint('/content/checkpoint_epoch_4.pt')

    for epoch in range(start, NUM_EPOCHS):
        model.train()
        margin.train()

        train_loss = []

        lr_backbone, lr_margin = step_lr(
            optimizer, base_lr_backbone, base_lr_margin,
            epoch, milestones=step_milestones, gamma=step_gamma
        )
        # bar config
        pbar = tqdm(
            enumerate(train_loader),
            total=len(train_loader),
            desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [LR: {lr_backbone:.6f}]"
        )

        for step, (inputs, targets) in pbar:
            inputs = inputs.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()

            # Forward
            outputs = model(inputs)
            outputs = F.normalize(outputs, p=2, dim=1)
            logits = margin(outputs, targets)
            loss = criterion(logits, targets)

            # Backward
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            torch.nn.utils.clip_grad_norm_(margin.parameters(), 5.0)
            optimizer.step()

            train_loss.append(loss.item())
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})

        train_loss = np.mean(train_loss)

        # ==================== EVALUATION ====================
        model.eval()
        margin.eval()

        test_loss = []
        embs = []
        labels = []

        with torch.no_grad():
            for inputs, targets in tqdm(test_loader, desc="Evaluating"):
                inputs = inputs.to(device)
                targets = targets.to(device)

                outputs = model(inputs)
                outputs = F.normalize(outputs, p=2, dim=1)
                logits = margin(outputs, targets)
                loss = criterion(logits, targets)
                test_loss.append(loss.item())

                embs.append(outputs.cpu())
                labels.append(targets.cpu())

        test_loss = np.mean(test_loss)
        eval_res = evaluate.evaluate(embs, labels, max_per_class=50, n_linspace=1000)

        # Get current learning rates
        current_lr_backbone = optimizer.param_groups[0]['lr']
        current_lr_margin = optimizer.param_groups[1]['lr']

        early.step(eval_res["tar_far_3"], model, margin)

        # helper function
        helper.save_checkpoint(model, margin, epoch, optimizer, train_loss, eval_res["tar_far_3"], eval_res["tar_far_4"])
        helper.print_results(optimizer, epoch, NUM_EPOCHS, train_loss, eval_res)

        # save scores
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        tarfar3s.append(eval_res["tar_far_3"])
        tarfar4s.append(eval_res["tar_far_4"])
        accs.append(eval_res["accuracy"])
        rocs.append(eval_res["roc_auc"])

        if early.should_stop:
            print("Training stopped early.")
            break

    print(f"\nTraining completed! Best TAR@FAR: {early.best_acc:.4f}")
    return train_losses, test_losses, tarfar3s, tarfar4s, accs, rocs

train()

==> Loading checkpoint from /content/checkpoint_epoch_4.pt
==> Resuming from epoch 4


Evaluating: 100%|██████████| 450/450 [02:10<00:00,  3.44it/s]


✓ Saved best model: TAR@FAR1e-3=0.7025
Saved checkpoint at epoch 5

Epoch 5/10
Learning Rate: Backbone=0.100000, Margin=0.500000
Train Loss: 7.9303
Eval Metrics:
  - Accuracy: 0.9382
  - ROC AUC: 0.9813
  - TAR@FAR1e-3: 0.7025
  - TAR@FAR1e-4: 0.5440
  - Threshold: 0.2361



Evaluating: 100%|██████████| 450/450 [02:10<00:00,  3.45it/s]


✓ Saved best model: TAR@FAR1e-3=0.8533
Saved checkpoint at epoch 6

Epoch 6/10
Learning Rate: Backbone=0.010000, Margin=0.050000
Train Loss: 2.8513
Eval Metrics:
  - Accuracy: 0.9620
  - ROC AUC: 0.9898
  - TAR@FAR1e-3: 0.8533
  - TAR@FAR1e-4: 0.7504
  - Threshold: 0.2251



...

Evaluating: 100%|██████████| 450/450 [02:10<00:00,  3.45it/s]


✓ Saved best model: TAR@FAR1e-3=0.8613
Saved checkpoint at epoch 9

Epoch 9/10
Learning Rate: Backbone=0.010000, Margin=0.050000
Train Loss: 2.1256
Eval Metrics:
  - Accuracy: 0.9617
  - ROC AUC: 0.9893
  - TAR@FAR1e-3: 0.8613
  - TAR@FAR1e-4: 0.7517
  - Threshold: 0.2214



Evaluating: 100%|██████████| 450/450 [02:10<00:00,  3.45it/s]


Saved checkpoint at epoch 10

Epoch 10/10
Learning Rate: Backbone=0.010000, Margin=0.050000
Train Loss: 2.0743
Eval Metrics:
  - Accuracy: 0.9606
  - ROC AUC: 0.9887
  - TAR@FAR1e-3: 0.8545
  - TAR@FAR1e-4: 0.7457
  - Threshold: 0.2218


Training completed! Best TAR@FAR: 0.8613


([np.float64(7.930322970363072),
  np.float64(2.8513244303567067),
  np.float64(2.406608359854562),
  np.float64(2.2295248148509432),
  np.float64(2.125596137700762),
  np.float64(2.0742738535744802)],
 [np.float64(44.7327838728163),
  np.float64(45.2183797454834),
  np.float64(44.854783113267686),
  np.float64(44.875245853000216),
  np.float64(44.73544124179416),
  np.float64(44.54028861999512)],
 [0.7025183421394313,
  0.853283564231831,
  0.854923167595529,
  0.8581698776796806,
  0.8612747842296744,
  0.8544682145901066],
 [0.5440056248735216,
  0.750354878115431,
  0.7534893270683745,
  0.7535469347541259,
  0.7516813320078641,
  0.74571524373221],
 [0.9381736591072581,
  0.9620387507699488,
  0.9617972416258366,
  0.9621118682172489,
  0.9616591308920476,
  0.9605660619829156],
 [0.9812521239776443,
  0.9897892773123033,
  0.9894308763450793,
  0.9896604356436439,
  0.9892771805529275,
  0.9886857709010143])